# **Spatial vectorization**

Boids is an artificial game that simulates the flocking behavior of birds. Boids is a short for bird-oid object. This is a good example that show spatial vectorization concept that is related to how a set of data share the same computation but data is dynamic and subgroups of data are only interacting with other subgroups of the same data, so in each iteration data is in a different state interacting with other subgroup 


In [2]:
import numpy as np

# Primer paso

Creamos los arrays que contendran a todos nuestros boids. Utilizamos 200 boids. Column 0 will be X data and column 1 will be Y data.

In [15]:

n = 5
velocity = np.zeros((n, 2), dtype=np.float32)
position = np.zeros((n, 2), dtype=np.float32)
position

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)

# Segundo paso

Calculamos las distancias con los vecinos. Es simple geometria. Lo que determina la distance entre los puntos es la hipotenusa, asi pues, la formula es **raiz((X2-X1)+(Y2-Y1))**.

Entonces, primero se crean matrices en las que cada una presenta X o Y restado por cada uno de los mismo X o Y en la matriz. ejemplo,

[x1,x2] - [x1,x2] = [
                    [x1-x1, x2-x1],
                    [x1-x2, x2-x2]
                                ]

Luego, una vez se haya restado, en la variable **distance** se calcula y almacena la hipotenusa, eso es lo que hace np.hypot(), hallar la hipotenusa entre x1 y y1.

***En la matriz resultante cada fila es un boid y su respectiva distancia con los demas boids.***

In [ ]:
dx = np.subtract.outer(position[:, 0], position[:, 0])
dy = np.subtract.outer(position[:, 1], position[:, 1])
distance = np.hypot(dx, dy)

# Tercer paso

Una vez calculadas las distancias de cada boid, definimos un radio. Abajo entramos que nos interesa aquellos que esten a una distancia menor de 25 y de 50, pero estos tienen que ser mayores de 0 porq el cero corresponde al boid itself. Ya entonces podemos sumar los vecinos que tenemos para cada boid, si el numero de vecinos es cero, se remplaza por 1, ya que mas adelante se usara este conteo en una division.

In [ ]:
# RADIO
mask_0 = (distance > 0)
mask_1 = (distance < 25)
mask_2 = (distance < 50)
mask_1 *= mask_0
mask_2 *= mask_0
mask_3 = mask_2

# SUM NEIGHBOURS
mask_1_count = np.maximum(mask_1.sum(axis=1), 1)
mask_2_count = np.maximum(mask_2.sum(axis=1), 1)
mask_3_count = mask_2_count

# Cuarto paso

Escribimos las reglas del juego:

***Alignment***: dirección hacia el rumbo medio de los miembros de la bandada local.

***Cohesion***: dirección para moverse hacia la posición media (centro de masa) de los miembros de la bandada local.

***Separation***: maniobra para evitar la aglomeración de compañeros de bandada locales.


In [ ]:
#ALIGNMENT
# Compute the average velocity of local neighbours
target = np.dot(mask, velocity)/count.reshape(n, 1)

# Normalize the result
norm = np.sqrt((target*target).sum(axis=1)).reshape(n, 1)
target *= np.divide(target, norm, out=target, where=norm != 0)

# Alignment at constant speed
target *= max_velocity

# Compute the resulting steering
alignment = target - velocity



In [ ]:
#COHESION

# Compute the gravity center of local neighbours
center = np.dot(mask, position)/count.reshape(n, 1)

# Compute direction toward the center
target = center - position

# Normalize the result
norm = np.sqrt((target*target).sum(axis=1)).reshape(n, 1)
target *= np.divide(target, norm, out=target, where=norm != 0)

# Cohesion at constant speed (max_velocity)
target *= max_velocity

# Compute the resulting steering
cohesion = target - velocity

In [ ]:
#SEPARATION

# Compute the repulsion force from local neighbours
repulsion = np.dstack((dx, dy))

# Force is inversely proportional to the distance
repulsion = np.divide(repulsion, distance.reshape(n, n, 1)**2, out=repulsion,
                        where=distance.reshape(n, n, 1) != 0)

# Compute direction away from others
target = (repulsion*mask.reshape(n, n, 1)).sum(axis=1)/count.reshape(n, 1)

# Normalize the result
norm = np.sqrt((target*target).sum(axis=1)).reshape(n, 1)
target *= np.divide(target, norm, out=target, where=norm != 0)

# Separation at constant speed (max_velocity)
target *= max_velocity

# Compute the resulting steering
separation = target - velocity